In [3]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

headers = {'User Agent':'Mozilla/5.0'}

players = [
    'kyrie-irving', 'derrick-williams-2', 'tristan-thompson-2', 'brandon-knight', 'kemba-walker',
    'jimmer-fredette', 'klay-thompson','alec-burks', 'markieff-morris', 'marcus-morris', 'kawhi-leonard', 
    'nikola-vucevic', 'iman-shumpert', 'chris-singleton','tobias-harris', 'nolan-smith', 'kenneth-faried', 
    'reggie-jackson', 'marshon-brooks', 'jordan-hamilton', 'jajuan-johnson', 'norris-cole', 'cory-joseph', 
    'jimmy-butler', 'justin-harper', 'kyle-singler', 'shelvin-mack', 'tyler-honeycutt', 'trey-thompkins',
    'chandler-parsons', 'jon-leuer', 'darius-morris', 'malcolm-lee', 'charles-jenkins', 'josh-harrellson', 
    'travis-leslie', 'keith-benson', 'josh-shelby', 'lavoy-allen', 'jon-diebler', 'vernon-macklin', 
    'deandre-liggins', 'etwaun-moore', 'isaiah-thomas']   

player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2011')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue

    try:
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2011'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/30))*2.95), 2)
        year_list.append(year)
        
    except:
        age_list.append(22)
        year_list.append(.85)
    
table.insert(2, "Age", age_list)
table.insert(2, "Year", year_list)
table.insert(0, "Name", playerlist)

table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

table["Player Grade"] = ((table['PTS']) + (table['TRB']*1.5) + (table['AST']*2) +
(table['BLK']*3) + (table['STL']*3) + (table['3P']*5)+ (table['FT%']*7) + (table['SOS']) + 
(table['3P%']*10)) * table['Year']
table["Player Grade"] = table["Player Grade"]*1.50
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['TOV']
del table['PF']
table

,Name,Season,School,Age,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,PTS,,Player Grade
1,Kyrie Irving,2010-11,Duke,19.3,8.32,11,27.5,.529,1.6,3.5,0.462,0.901,3.4,4.3,1.5,0.5,17.5,,101.5
2,Kemba Walker,2010-11,UConn,21.2,10.44,41,37.6,.428,1.8,5.5,0.330,0.819,5.4,4.5,1.9,0.2,23.5,,98.4
3,Brandon Knight,2010-11,Kentucky,19.6,8.91,38,35.9,.423,2.3,6.1,0.377,0.795,4.0,4.2,0.7,0.2,17.3,,98.1
4,Klay Thompson,2010-11,Washington State,21.4,6.81,34,34.7,.436,2.9,7.2,0.398,0.838,5.2,3.7,1.6,0.9,21.6,,96.2
5,Marshon Brooks,2010-11,Providence,22.0,6.44,32,36.5,.483,2.1,6.2,0.340,0.772,7.0,2.5,1.5,1.2,24.6,,94.3
6,Derrick Williams,2010-11,Arizona,20.1,7.99,38,30.0,.595,1.1,1.9,0.568,0.746,8.3,1.1,1.0,0.7,19.5,,92.6
7,Jimmer Fredette,2010-11,BYU,22.4,5.76,37,35.8,.452,3.4,8.5,0.396,0.894,3.4,4.3,1.3,0.0,28.9,,89.4
8,Kawhi Leonard,2010-11,San Diego State,20.0,6.36,36,32.6,.444,0.7,2.4,0.291,0.759,10.6,2.5,1.4,0.6,15.5,,88.9
9,Tobias Harris,2010-11,Tennessee,19.0,8.73,34,29.2,.460,0.7,2.2,0.303,0.753,7.3,1.3,0.7,0.9,15.3,,87.8
10,Reggie Jackson,2010-11,Boston College,21.2,7.31,34,34.1,.503,2.1,5.0,0.420,0.796,4.3,4.5,1.1,0.5,18.2,,86.2
